<a href="https://colab.research.google.com/github/MaQuest/Summer2021/blob/main/bert_reco_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install sentence-transformers
!pip install tweepy
!pip install bert-extractive-summarizer
!pip install nltk
!pip install google-cloud-vision

     |████████████████████████████████| 81kB 4.2MB/s 
     |████████████████████████████████| 2.3MB 10.5MB/s 
     |████████████████████████████████| 1.2MB 22.9MB/s 
     |████████████████████████████████| 901kB 51.3MB/s 
     |████████████████████████████████| 3.3MB 48.9MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.2.0-cp37-none-any.whl size=123339 sha256=1df5eb8dd7da89bd71c4aecf2a079fd6cc2e5173f6e319ce631970bb87638098
  Stored in directory: /root/.cache/pip/wheels/0f/06/f7/faaa96fdda87462b4fd5c47b343340e9d5531ef70d0eef8242
Successfully built sentence-transformers
     |████████████████████████████████| 471kB 5.2MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 


In [3]:
import pandas as pd
import json
import os
from sentence_transformers import SentenceTransformer, util
from google.colab import drive
import tweepy
import re
import os,io
from google.cloud import vision
from google.cloud.vision_v1 import types
import pandas as pd
import nltk

ACCESS_TOKEN="1401579178072231936-98mY1wOw1UR3GHOjdo4ePnfdfUHt6n"
ACESS_TOKEN_SECRET="QiIslicgx8GNGrGDPrx8xNZc2WoCyrUsZumgXSyGzGLOx"
CONSUMER_KEY="uaiStuv7EYdYxWSomPKHvSSF5"
CONSUMER_SECRET="JOQryy37w9HPMSrSw8msyyb048iqeHmK4xCRyWP1oBhLKwLYlb"

auth=tweepy.OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN,ACESS_TOKEN_SECRET)
api=tweepy.API(auth)

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'seismic-diorama-316110-5569927e0d86.json'
client = vision.ImageAnnotatorClient()

In [4]:
def pre_process(tweets):

    for i in range(0, len(tweets)):

        if (tweets[i] is not None):

            if(tweets[i]!=tweets[i]):
                tweets[i]=""

            tweets[i] = tweets[i].lower()  # To lower case
            tweets[i] = tweets[i].replace('@','')  # remove @
            tweets[i] = tweets[i].replace('#','')  # remove #
            tweets[i] = remove_urls(tweets[i])  # remove URL
            tweets[i] = remove_emojis(tweets[i])  # remove emojis
            tweets[i] = "".join(j for j in tweets[i] if j not in (
            "?", ".", ";", ":", "!", "-", ",", "[", "]", "(", ")", "’", "‘", '"', "$", "'", "“", "”", "•", "=", "+",
            "%", "/", "&", "|", "~"))  # remove punctuations
            tweets[i] = removeNonEnglishWordsFunct(tweets[i])

    return tweets

def remove_urls (str):

    str = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', str, flags=re.MULTILINE)
    return(str)


def remove_emojis(data):

    emoji = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoji, '', data)

def removeNonEnglishWordsFunct(x):

    new_string=re.sub('[^a-zA-Z0-9]',' ',x)

    cleaned_string=re.sub('\s+',' ',new_string)
    return cleaned_string

In [73]:
df = pd.read_csv('ccc-organizations-2011_1.csv')

charities =df.OrganizationName.values+" "+df.Description.values
cities=df.City.values

print(len(charities), "Charities")

#We then load the allenai-specter model with SentenceTransformers
model = SentenceTransformer('allenai-specter')

#To encode the descriptions to a single string
charity_texts = [charity for charity in charities]
cities_texts=[city for city in cities]

#Compute embeddings for all descriptions
corpus_embeddings = model.encode(charity_texts, convert_to_tensor=True)


318 Charities


In [97]:

def search_papers_location(location):

   results=[]
   for i in range(0,len(cities_texts)):
     if(cities_texts[i]==location):
       results.append(i)
   return results

  

In [140]:
import random
from termcolor import colored

#Generate 5 random numbers between 10 and 30
priority = random.sample(range(0, 319), 40)
print(priority)


[122, 117, 254, 238, 233, 10, 291, 48, 157, 267, 162, 252, 80, 35, 108, 242, 137, 283, 220, 240, 276, 140, 226, 116, 310, 245, 85, 78, 75, 210, 79, 96, 318, 294, 239, 31, 270, 180, 160, 223]


In [153]:
def generate_priority(title):

   query_embedding = model.encode(title+'[SEP]', convert_to_tensor=True) # Converts to tensor
   search_hits = util.semantic_search(query_embedding, corpus_embeddings,top_k=10)
   search_hits = search_hits[0]
   
   count = 0
   top_header = "\n\nPriority Charities\n\n"
   print(colored(top_header,'yellow'))

   flag=0

   for hit in search_hits:

          for i in range(0,len(priority)):
            if(hit['corpus_id']==priority[i]):
              flag=1
              break

          if(flag==1 and search_hits[count]['score']>0.60):
            related_charities = charities[hit['corpus_id']]
            count += 1
            subsetDataFrame = df[df['OrganizationName']+" "+df['Description']== related_charities]
            k=subsetDataFrame.values
            print("\n"+str(count)+") "+colored(str(k[0][1]), 'red'))
            print("similiarity score of " + str(format(search_hits[count-1]['score'],".2f")))
            print("Description of charity : " +related_charities+" "+cities_texts[hit['corpus_id']])
            flag=0
            count+=1

generate_priority("cancer")




Priority Charities



1) Children's Cancer Foundation, Inc. (The)
similiarity score of 0.84
Description of charity : Children's Cancer Foundation, Inc. (The) Provides grants for building facilities and research to hospitals in the Baltimore/Washington DC areas for pediatric oncology. Owings Mills


In [151]:

def search_papers(title,city_indices,status):

   query_embedding = model.encode(title+'[SEP]', convert_to_tensor=True) # Converts to tensor
   search_hits = util.semantic_search(query_embedding, corpus_embeddings,top_k=10)
   search_hits = search_hits[0]
   
   count = 0
   flag=0
   
   top_header = "\n\nTop 10 related charities\n\n"
   print(colored(top_header,'blue'))


   for hit in search_hits:

        if(status==0) :

          for i in range(0,len(city_indices)):
            if(hit['corpus_id']==city_indices[i]):
              flag=1
              break

          if(flag==1):

            related_charities = charities[hit['corpus_id']]
            count += 1
            subsetDataFrame = df[df['OrganizationName']+" "+df['Description']== related_charities]
            k=subsetDataFrame.values
            print("\n"+str(count)+") "+colored(str(k[0][1]), 'red'))
            print("similiarity score of " + str(format(search_hits[count-1]['score'],".2f")))
            print("Description of charity : " +related_charities+" "+cities_texts[hit['corpus_id']])
            flag=0

        else:

          related_charities = charities[hit['corpus_id']]
          count += 1
          subsetDataFrame = df[df['OrganizationName']+" "+df['Description']== related_charities]
          k=subsetDataFrame.values
          print("\n"+str(count)+") "+colored(str(k[0][1]), 'red'))
          print("similiarity score of " + str(format(search_hits[count-1]['score'],".2f")))
          print("Description of charity : " +related_charities+" "+cities_texts[hit['corpus_id']])
          flag=0


#want to make a big impact donations made through birdies4kids for the alberta diabetes foundation are matched up to 50  birdies for kids runs until august 15th 2021  to donate and read more 

#alberta wins reddeer sisters donate land to nature conservancy of canada ncc preserve a haven for species wildlife ruth dorothy bower donate 193 hectares of land on the west bank of the reddeer river bowerwildlifesanctuary

#in the news the kidney foundation encourages other canadian organizations and companies to to adopt policies to support living organdonors read more about kidneycanada s wage replacement policy for living organ and tissue donation

#participate in the scotiabank calgary marathon charity challenge to help children amp families at childrens cottage society theres an event for everyone of all abilities register or donate at  thanks for your consideration children families 

#can you help us many children need their very own books strong reading role models amp safe fun reading spaces your donation helps more children experience the magic of reading amp a lifetime of opportunity 

In [155]:
print("What basis do you want to generate a recommendation on ? ")
print("\n1. Enter a sentence  ")
print("2. Trending tweet on a particular hashtag ")
print("3. Image-based recommendation generation \n")
choice=input("Enter your choice : ")

if(choice=="1"):

  title=input("\nEnter a sentence : ")
  enable_loc=str(input("Do you wish to enable location ? "))
  if(enable_loc=="yes"):
    status=0
    location=str(input("Enter location : "))
    loc_indices=search_papers_location(location)
    generate_priority(title)
    search_papers(title,loc_indices,status)
  else:
    temp=[]
    status=1
    generate_priority(title)
    search_papers(title,temp,status)

elif(choice=="2"):

  tag = input("\nEnter a hashtag : ")
  tweets=tweepy.Cursor(api.search,q=tag,result_type='popular',tweet_mode="extended").items(1)
  temp=[]
  for tweet in tweets :

      temp.append(tweet.full_text)

  temp=pre_process(temp)
  tweet=temp[0]
  print("\nMost famous tweet : \n"+colored(tweet,'red'))
  search_papers(tweet)

elif(choice=="3"):

  with io.open("horse.jpg", 'rb') as image_file:
    content = image_file.read()


  sample_tweet = """Beautiful Rose (R) and beautiful Sunflower (L) are seen here enjoying a breezy afternoon here at the ranch. 

  Sparkleshttp://linktr.ee/PRRHR

  #miniturehorses #horse #rescue #minihorsesoftwitter #BestFriendsDay #donate #tuesdayvibe"""

  sample_tweet=pre_process([sample_tweet])

  image = vision.Image(content=content)

  response_label = client.label_detection(image=image)
  response_text = client.text_detection(image=image)

  temp=""
  count=0

  for label in response_label.label_annotations:
      if(count<3):
        temp+=label.description+" "
        count+=1

  for r in response_text.text_annotations:
      temp+=r.description+" "


  temp=pre_process([temp])
  print("\n\nInterpretation of tweet : "+colored(sample_tweet[0]+" "+temp[0],'red'))
  search_papers(sample_tweet[0]+" "+temp[0])

What basis do you want to generate a recommendation on ? 

1. Enter a sentence  
2. Trending tweet on a particular hashtag 
3. Image-based recommendation generation 

Enter your choice : 1

Enter a sentence : alberta wins reddeer sisters donate land to nature conservancy of canada ncc preserve a haven for species wildlife ruth dorothy bower donate 193 hectares of land on the west bank of the reddeer river bowerwildlifesanctuary
Do you wish to enable location ? no


Priority Charities



1) Sierra Club Foundation
similiarity score of 0.90
Description of charity : Sierra Club Foundation Preserves wilderness and protects environmental quality worldwide through a powerful combination of education, scientific research, publishing and litigation. San Francisco


Top 10 related charities



1) Natural Resources Defense Council
similiarity score of 0.90
Description of charity : Natural Resources Defense Council Defends embattled wilderness, rivers, clean air, coasts and wildlife across America